In [12]:
import torch
import pandas as pd
import numpy as np 
import random

device = "cpu"

text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([1680, 768])

In [13]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,"[0.0674242675, 0.0902280286, -0.00509552006, -..."
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,"[0.0552156419, 0.0592138022, -0.0166167412, -0..."
2,-37,Contents Preface University of Hawai‘i at Māno...,766,116,191.50,"[0.0279801395, 0.0339813568, -0.0206426941, 0...."
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,144,235.25,"[0.0682566836, 0.0381274521, -0.00846857857, -..."
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,"[0.033026401, -0.00849775691, 0.00957154203, -..."


In [14]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device) # choose the device to load the model to

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Can use dot product or cosine similarity if outputs of model aren't normalized

In [15]:
query = "why is protien important?"
print(f"Query: {query}")

#always embed query with the same embedding model as your data 
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

#top-k 
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: why is protien important?
Time take to get scores on 1680 embeddings: 0.00017 seconds.


torch.return_types.topk(
values=tensor([0.5791, 0.5456, 0.5380, 0.5327, 0.5239]),
indices=tensor([601, 551, 584, 547,  52]))

In [16]:
pages_and_chunks[601]

{'page_number': 405,
 'sentence_chunk': 'Image by Braden Collum on unsplash.co m / CC0 Proteins in a Nutshell UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  Proteins are long chains of amino acids folded into precise structures that determine their functions, which are in the tens of thousands. They are the primary construction materials of the body serving as building blocks for bone, skin, hair, muscle, hormones, and antibodies. Without them we cannot breakdown or build macromolecules, grow, or heal from a wound. Too little protein impairs bodily functions and too much can lead to chronic disease. Eat proteins in moderation, at least 10 percent of the calories you take in and not more than 35 percent. Proteins are in a variety of foods. More complete sources are in animal-based foods, but choose those low in saturated fat and cholesterol. Some plant-based foods are also complete protein sources and don’t add much to your saturated